In [3]:
import cv2
import numpy as np
import cv2
import time
from IPython.display import clear_output
import math
from IPython.display import clear_output
    
#WIND_X = 480 
#WIND_Y = 360

WIND_X = 640
WIND_Y = 480

def get_available_cameras (upper_bound = 10, lower_bound = 0):
    available = []
    
    for i in range (lower_bound, upper_bound):
        cap = cv2.VideoCapture (i)
    
        if (cap.isOpened ()):
            available.append (i)
    
        cap.release ()
    
    return available

def turn_to_RG (img):
    (h, w, d) = img.shape
    
    norm = np.zeros ((h, w), np.float)
    norm = img [:, :, 0].astype ('float') +\
           img [:, :, 1].astype ('float') +\
           img [:, :, 2].astype ('float')
    
    norm [norm == 0] = 5
    
    turned = np.zeros (img.shape, np.uint8)
    turned [:, :, 0] = ((img [:, :, 0].astype ('float')) / norm * 255).astype ('uint8')
    turned [:, :, 1] = ((img [:, :, 1].astype ('float')) / norm * 255).astype ('uint8')
    turned [:, :, 2] = ((img [:, :, 2].astype ('float')) / norm * 255).astype ('uint8')
    
    return turned

def obtain_color_ratio_mask (img, components, th, bl, use_rg = False, use_morph = False):
    sh = img [:, :, 0].shape
    
    rg = img
    
    if (use_rg == True):
        rg = turn_to_RG (img)
    
    smoothed = cv2.blur (rg, (bl, bl))
    
    needed = img.copy ()
    needed [:, :, 0] = np.full (sh, components [0])
    needed [:, :, 1] = np.full (sh, components [1])
    needed [:, :, 2] = np.full (sh, components [2])
    
    diff = cv2.absdiff (smoothed, needed)
    
    dif = diff [:, :, 1]# + diff [:, :, 1] + diff [:, :, 2]
    
    cv2.imshow ("fok", dif)
    
    ret, res_mask = cv2.threshold (dif, th, 255, cv2.THRESH_BINARY_INV)
    
    #res_mask = cv2.adaptiveThreshold (dif, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, 51, 10)
    
    if (use_morph == True):
        res_mask = cv2.morphologyEx (res_mask, cv2.MORPH_ERODE, np.ones ((int (bl), int (bl)), np.uint8))
        #res_mask = cv2.morphologyEx (res_mask, cv2.MORPH_OPEN, np.ones ((int (bl), int (bl)), np.uint8))
    
    cv2.imshow ("kek", res_mask)
    
    res = cv2.bitwise_and (img, img, mask = res_mask)
    
    return res, res_mask

def find_max_bounding_box (mask):
    result = np.array (mask)
    output = cv2.connectedComponentsWithStats (mask, 8, cv2.CV_32S)
    labels_num = output      [0]
    labels     = output      [1]
    stats      = output      [2]
    sz         = stats.shape [0]
    
    max_w     = 0
    max_label = 0
    
    for label_num in range (1, sz - 1):
        if (stats [label_num, cv2.CC_STAT_WIDTH] > max_w):
            max_w = stats [label_num, cv2.CC_STAT_WIDTH]
            max_label = label_num
    
    top    = stats [max_label, cv2.CC_STAT_TOP]
    left   = stats [max_label, cv2.CC_STAT_LEFT]
    width  = stats [max_label, cv2.CC_STAT_WIDTH]
    height = stats [max_label, cv2.CC_STAT_HEIGHT]
    
    return (left, top), (left + width, top + height)

def find_low_middle_point (mask):
    result = np.array (mask)
    output = cv2.connectedComponentsWithStats (mask, 8, cv2.CV_32S)
    labels_num = output      [0]
    labels     = output      [1]
    stats      = output      [2]
    sz         = stats.shape [0]
    
    max_w     = 0
    max_label = 0
    
    for label_num in range (1, sz - 1):
        if (stats [label_num, cv2.CC_STAT_WIDTH] > max_w):
            max_w = stats [label_num, cv2.CC_STAT_WIDTH]
            max_label = label_num
    
    top    = stats [max_label, cv2.CC_STAT_TOP]
    left   = stats [max_label, cv2.CC_STAT_LEFT]
    width  = stats [max_label, cv2.CC_STAT_WIDTH]
    height = stats [max_label, cv2.CC_STAT_HEIGHT]
    
    print ("wid", width)
    
    return (left + int (width / 2), top + height)

def find_goal (img):
    #goal1, g_mask1 = obtain_color_ratio_mask (img, (245, 245, 245), 40, 3, use_morph = True)    
    #goal2, g_mask2 = obtain_color_ratio_mask (img, (85, 85, 85), 15, 1, use_rg = True, use_morph = True)    
    
    #goal1, g_mask1 = obtain_color_ratio_mask (img, (220, 255, 220), 30, 3, use_morph = True)    
    goal1, g_mask1 = obtain_color_ratio_mask (img, (220, 235, 220), 20, 3, use_morph = True)    
    
    #goal1, g_mask1 = obtain_color_ratio_mask (img, (180, 220, 180), 30, 3, use_morph = True)    

    #goal2, g_mask2 = obtain_color_ratio_mask (img, (85, 85, 85), 13, 1, use_rg = True, use_morph = True)    
    
    #cv2.imshow ("kek", goal1)
    #cv2.imshow ("kek2", goal2)
    
    #goal_mask_res = cv2.bitwise_and (g_mask1, g_mask2)
    #goal_res = cv2.bitwise_and (img, img, mask = goal_mask_res)

    goal_mask_res = g_mask1
    goal_res = cv2.bitwise_and (img, img, mask = goal_mask_res)
    
    tl, br = find_max_bounding_box (goal_mask_res)
    return tl, br

def find_goal_middle (img):
    #goal1, g_mask1 = obtain_color_ratio_mask (img, (245, 245, 245), 40, 3, use_morph = True)    
    #goal2, g_mask2 = obtain_color_ratio_mask (img, (85, 85, 85), 15, 1, use_rg = True, use_morph = True)    
    
    #goal1, g_mask1 = obtain_color_ratio_mask (img, (220, 250, 220), 30, 3, use_morph = True)    
    
    goal1, g_mask1 = obtain_color_ratio_mask (img, (220, 235, 220), 20, 3, use_morph = True)    
    
    #goal2, g_mask2 = obtain_color_ratio_mask (img, (85, 85, 85), 13, 1, use_rg = True, use_morph = True)    
    
    #cv2.imshow ("kek", goal1)
    #cv2.imshow ("kek2", goal2)
    
    #goal_mask_res = cv2.bitwise_and (g_mask1, g_mask2)
    #goal_res = cv2.bitwise_and (img, img, mask = goal_mask_res)

    goal_mask_res = g_mask1
    goal_res = cv2.bitwise_and (img, img, mask = goal_mask_res)
    
    mid = find_low_middle_point (goal_mask_res)
    return mid

#адаптивный трешхолд
#функция, на вход картинка на выход координаты середины
#написать, что надо застелить окрестность ворот
#

In [6]:
#"""
cam = cv2.VideoCapture (1)
#cam = cv2.VideoCapture ("inp.webm")

#cv2.namedWindow  ("field", cv2.WINDOW_NORMAL)
#cv2.resizeWindow ("field", (WIND_X, WIND_Y))

output_shape = (WIND_X * 3, WIND_Y * 2)

fourcc = cv2.VideoWriter_fourcc (*'XVID')
#out    = cv2.VideoWriter ('output.avi', fourcc, 20.0, output_shape)
#out    = cv2.VideoWriter ('output.mp4', fourcc, 20.0, output_shape)

cv2.namedWindow  ("goal detection", cv2.WINDOW_NORMAL)
cv2.resizeWindow ("goal detection", output_shape)

while (cam.isOpened ()):
    ret1, frame = cam.read ()
    
    if (ret1 != True):
        break
    
    frame_resized = cv2.resize (frame, (int (WIND_X), int (WIND_Y)))
    
    cv2.waitKey (1)
    
    tl, br = find_goal (frame_resized)
    goal_on_inp = cv2.rectangle (frame_resized.copy (), tl, br, (255, 0, 0), 5)
    
    ###mid = find_goal_middle (frame_resized)
    ###goal_on_inp_ = cv2.circle (frame_resized.copy (), mid, 5, (250, 10, 253), -1)
    
    goal_marked = cv2.addWeighted (frame_resized, 0.6, goal_on_inp, 0.4, 0)
    ###goal_marked = cv2.addWeighted (goal_marked, 0.6, goal_on_inp_, 0.4, 0)
    
    #output_image = np.concatenate ((frame_resized, goal_res), axis = 1)
    output_image = np.concatenate ((frame_resized, goal_marked), axis = 1)
    
    cv2.imshow ("goal detection", output_image)
    #out.write (output_image)
    
    #x0 =  1        #x coordinate of the point on the screen, pixels
    #y0 = -1.3      #y coordinate of the point on the screen, pixels
    
    #x0 = (tl [0] + br [0]) / 2 - WIND_X / 2
    #y0 = - tl [1] + WIND_Y / 2
    #k1 =  0      #incline  angle.               from  0 t0 1
    #k2 =  0      #rotation around Z axis angle. from -1 to 1
    
    #A = point_position(k1, k2, x0, y0)
    #B = ball_position(k1, k2, x0, y0)
    #clear_output (wait = True)
    #print (A)
    #print (x0, y0)
    
    time.sleep  (0.01)
    
    if (cv2.waitKey (1) & 0xFF == ord ('q')):
        break

cam.release ()
#out.release ()

cv2.waitKey           (1)
cv2.destroyAllWindows ()
#"""

In [5]:
output_shape = (WIND_X * 3, WIND_Y * 2)

cv2.namedWindow  ("goal detection", cv2.WINDOW_NORMAL)
cv2.resizeWindow ("goal detection", output_shape)

frame = cv2.imread ("5.jpg")

frame_resized = cv2.resize (frame, (int (WIND_X), int (WIND_Y)))

#tl, br = find_goal (frame_resized)
mid = find_goal_middle (frame_resized)

print ("mid", mid [0] - WIND_X / 2, - mid [1] + WIND_Y / 2)

#goal_on_inp = cv2.rectangle (frame_resized.copy (), tl, br, (255, 0, 0), 5)
goal_on_inp = cv2.circle (frame_resized.copy (), mid, 8, (250, 10, 253), -1)

goal_marked = cv2.addWeighted (frame_resized, 0.6, goal_on_inp, 0.7, 0)

output_image = np.concatenate ((frame_resized, goal_marked), axis = 1)

cv2.imshow ("goal detection", output_image)

#x0 = (tl [0] + br [0]) / 2 - WIND_X / 2
#y0 = - tl [1] + WIND_Y / 2

#k1 =  0      #incline  angle.               from  0 t0 1
#k2 =  0      #rotation around Z axis angle. from -1 to 1

#A = point_position (k1, k2, x0, y0)

#print (A)
#print (x0, y0)

time.sleep  (0.01)

cv2.waitKey           (0)

cv2.destroyAllWindows ()

wid 214
mid -60.0 -141.0


In [ ]:
#3: mid -5.0 -195.0    # 360
#4: mid -16.0 -147.0   # 263
#5: mid -60.0 -141.0   # 214
#6: mid 209.0 -240.0 (неправильно :( )

In [ ]:
def position_calc(k1, k2, x0, y0):
    """
    Makes necessary calculations
    
    Arguments:
        k1 - rotation around Z axis angle. from -1 to 1
        k2 - incline  angle.               from  0 t0 1
        x0 - x coordinate of the point on the screen, pixels
        y0 - y coordinate of the point on the screen, pixels
        
    """   
    
    pi  = np.pi
    cos = math.cos
    sin = math.sin


    ###__constants__####

    #h0 =  80  #45.959                    #height of the neck joint, cm
    
    h0 =  45.959                    #height of the neck joint, cm
    h2 =  5.071                     #position of the photomatrix relative to the neck joint along the Y axis, cm
    h3 =  6.364                     #position of the photomatrix relative to the neck joint along the Z axis, cm
    f  =  600                       #photomatrix-lens distanse, px


    e1 =  np.array([0, 0, 1])              #Z axis vector 
    e2 =  np.array([1, 0, 0])              #X axis vector
    H  =  h0 * e1                          #neck joint vector
    L0 =  np.array([0, h2, h3])            #position of the lens reletive to neck joint vector
    q0 = -np.array([x0, f, y0])            #position of the point on the photomatrix vector reletive to lens
    Q0 =  L0 + q0                          #position of the point on the photomatrix vector reletive to neck joint



    def rotation_matrix(axis, theta):
        """
        Return the rotation matrix associated with counterclockwise rotation about
        the given axis by theta radians.
        """
        axis = np.asarray(axis)
        axis = axis / math.sqrt(np.dot(axis, axis))
        a = math.cos(theta / 2.0)
        b, c, d = -axis * math.sin(theta / 2.0)
        aa, bb, cc, dd = a * a, b * b, c * c, d * d
        bc, ad, ac, ab, bd, cd = b * c, a * d, a * c, a * b, b * d, c * d
        return np.array([[aa + bb - cc - dd, 2 * (bc + ad), 2 * (bd - ac)],
                         [2 * (bc - ad), aa + cc - bb - dd, 2 * (cd + ab)],
                         [2 * (bd + ac), 2 * (cd - ab), aa + dd - bb - cc]])


    a1 =  pi/2 * k1                  #rotation around Z axis angle
    a2 = -pi/2 * k2                  #incline  angle

    R1 = rotation_matrix(e1, a1)     #rotaion around Z axis matrix
    R2 = rotation_matrix(e2, a2)     #incline up-down matrix
    R  = np.matmul(R1, R2)           #full rotation matrix


    Q = R.dot(Q0) + H                #position of the point on the screen reletive to floor vector
    q = R.dot(q0)                    #ligth ray vector
    
    return [Q, q]

def point_position(k1, k2, x0, y0):
    """
    Takes same args as calc_position
    Returns the position of the point on the floor reletive to robot
    """
    calc = position_calc(k1, k2, x0, y0)   #takes calculations from position_calc
    
    Q = calc[0]                            #position of the point on the screen reletive to floor vector
    q = calc[1]                            #ligth ray vector
    
    A = Q - q * Q[2] / q[2]                #position of the point on the floor reletive to robot
    
    return A

def ball_position(k1, k2, x0, y0):
    """
    Takes same args as calc_position
    Returns the position of the ball on the floor reletive to robot
    """    
    r = 5                                  #radius of the ball
    calc = position_calc(k1, k2, x0, y0)   #takes calculations from position_calc
    
    Q = calc[0]                            #position of the point on the screen reletive to floor vector
    q = calc[1]                            #ligth ray vector
    
    
    B = Q - q * (Q[2] - r) / q[2]          #position of the ball on the floor reletive to robot
    B[2] = 0                               #on the floor z = 0
    
    return B

In [46]:
a = []
b = []
c = []

def calc (A, B, C):
    P = 70 - 0.3 * (A + B + C)

    PrA = A * (P - 1)
    PrB = B * (P - 1)
    PrC = C * (P - 1)

    a.append (PrA)
    b.append (PrB)
    c.append (PrC)
    
    print ("Our profit:", PrB)
    print ("A pr", PrA)
    print ("C pr", PrC)
    
    print ("World profit:", (A + B + C) * (P - 1))
    print ("Total production:", A + B + C)

In [57]:
calc (42, 34.5, 45) #round 1
print ("\n")

calc (44, 70, 50) #round 2
print ("\n")

calc (46, 75, 65) #round 3
print ("\n")

calc (45, 50, 45) #round 4
print ("\n")

#calc (75, 75, 75) #round 4
#print ("\n")

#calc (38.3, 38.3, 38.3) #round 4
#print ("\n")

calc (70, 65, 75) #round 4
print ("\n")

print ("Summary:")
print (sum (a), sum (b), sum (c))

Our profit: 1122.9750000000001
A pr 1367.1000000000001
C pr 1464.7500000000002
World profit: 3954.8250000000007
Total production: 121.5


Our profit: 1386.0000000000002
A pr 871.2000000000002
C pr 990.0000000000002
World profit: 3247.2000000000007
Total production: 164


Our profit: 990.0000000000002
A pr 607.2000000000002
C pr 858.0000000000002
World profit: 2455.2000000000007
Total production: 186


Our profit: 1350.0
A pr 1215.0
C pr 1215.0
World profit: 3780.0
Total production: 140


Our profit: 390.0
A pr 420.0
C pr 450.0
World profit: 1260.0
Total production: 210


Summary:
44105.29799999998 50947.24799999999 48556.99800000001


In [50]:
175 - 55

120